In [130]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

url = 'https://coinmarketcap.com/historical/20191103/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/115.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [135]:
def scrape_data(start_date, end_date, tickers):
    dates = pd.date_range(start=start_date, end=end_date, freq='W-SUN').strftime('%Y%m%d').tolist()
    all_data = pd.DataFrame()
    for date in tqdm(dates):
        try:
            data = scrape_market_cap(date, tickers)
        except:
            print('Error trying to scrape data')
            data = pd.DataFrame()
        all_data = pd.concat([all_data, data])
    
    all_data['date'] = pd.to_datetime(all_data['date'])
    all_data = all_data.pivot(index='date', columns='symbol', values='Market Cap (USD)')
    
    return all_data

def scrape_market_cap(date, tickers):
    """
    Scrape market ap data for a given snapshot date from CoinMarketCap.
    'date' should be in 'YYYYMMDD' format.
    'tickers' is a list of ticker symbols to filter.
    """
    url = f'https://coinmarketcap.com/historical/{date}/'
    headers = {'User-Agent': 'Mozilla/5.0'}  # Use a proper user-agent to avoid request issues
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve data for {date}")

    soup = BeautifulSoup(response.content, 'html.parser')
    soup_str = str(soup)
    # Define your markers (adjust if necessary)
    start_marker = '{\\"data\\":['
    end_marker = '}}}]'
    # Find the start and end positions
    start_index = soup_str.find(start_marker)
    end_index = soup_str.find(end_marker, start_index)
    
    json_str = soup_str[start_index:end_index+len(end_marker)] + '}'
    fixed_str = json_str.encode('utf-8').decode('unicode_escape')
    data_dict = json.loads(fixed_str)

    df = pd.DataFrame(data_dict['data'])
    df['Market Cap (USD)'] = pd.DataFrame(pd.DataFrame(df['quote'].values.tolist())['USD'].values.tolist())['marketCap']
    filtered_df = df[df['symbol'].isin(tickers)][['symbol', 'Market Cap (USD)']]
    filtered_df['date'] = date

    return filtered_df
    

In [136]:
dates = ['20191103', '20191110']
tickers = ['BTC', 'ETH', 'XRP', 'BAT', 'DOGE']

all_data = scrape_data('20191103', '2019-12-24', tickers)
all_data

100%|██████████| 8/8 [00:01<00:00,  7.99it/s]


symbol,BAT,BTC,DOGE,ETH,XRP
date,,,,,
2019-11-03,3.140353e+08,1.664953e+11,3.213849e+08,1.977735e+10,1.259646e+10
2019-11-10,3.345501e+08,1.633644e+11,3.385345e+08,2.055912e+10,1.213239e+10
2019-11-17,3.768971e+08,1.548618e+11,3.232915e+08,2.010265e+10,1.148505e+10
2019-11-24,2.824466e+08,1.273274e+11,2.890364e+08,1.552346e+10,9.663858e+09
2019-12-01,2.675509e+08,1.342151e+11,2.802606e+08,1.644365e+10,9.756884e+09
2019-12-08,2.587016e+08,1.368474e+11,2.738550e+08,1.646473e+10,9.974529e+09
2019-12-15,2.614693e+08,1.294814e+11,2.637928e+08,1.558912e+10,9.467680e+09
2019-12-22,2.451052e+08,1.360880e+11,2.592720e+08,1.442865e+10,8.532973e+09
